In [ ]:
from web3 import Web3
import time
import subprocess
import sys
from google.cloud import storage
import os
import pandas as pd
from datetime import datetime
import threading
from optimizer_and_uploader import optimize_and_upload

In [ ]:
RPC_FILE = "config/rpc.txt"
GOOGLE_CREDENTIALS_FILE = 'config/ethereum-data-nero.json'
CONTRACT_ADDRESS = "0x2f1C2F0149b7da5Cb5Cf2921Fff7E8B44886530F"

announcement_print = "{:>2} | {:>32} | {:>32} | {:>32} | {:>10} | {:>32}"

In [ ]:
def init():
    with open(RPC_FILE, "r") as file:
        rpc = file.read().strip()
    w3 = Web3(Web3.HTTPProvider(rpc))
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = GOOGLE_CREDENTIALS_FILE
    client = storage.Client()
    return w3, client

def load_contract_abi():
    with open("config/contract_abi.txt", "r") as file:
        abi = file.read().strip()
    return abi.replace("\n", "").replace("\t", "")

def load_contract_bytecode():
    with open("config/contract_bytecode.txt", "r") as file:
        abi = file.read().strip()
    return abi.replace("\n", "").replace("\t", "")

def create_filters(messenger):
    announcement_filter = messenger.events.Announcement.createFilter(
        fromBlock="latest",  
        address=CONTRACT_ADDRESS
    )

    staking_filter = messenger.events.StakeDeposit.createFilter(
        fromBlock="latest",  
        address=CONTRACT_ADDRESS
    )

    withdrawing_filter = messenger.events.StakeWithdrawal.createFilter(
        fromBlock="latest",  
        address=CONTRACT_ADDRESS
    )
    
    return [announcement_filter, staking_filter, withdrawing_filter]

def now():
    return datetime.now().strftime('%B-%d %H:%M:%S ')

In [ ]:
def get_csv_head_and_content(file):
    if file == "stakers.csv":
        content = '{},{},{}'
        header = "address,totalStaked,blockNumber\n"
        
    if file == "withdrawers.csv":
        content = '{},{},{},{}'
        header = "address,receiver,amount,blockNumber\n"
        
    if file == "announcements.csv":
        content = '{},{},0x{},0x{},{},{}'
        header = "schemeID,stealthAddress,ephemeralPubKey,metadata,blockNumber,sender\n"
    
    return header, content

def create_file_if_not_existing(file, header):
    if not os.path.isfile(file):
        with open(file, "a") as file:
            file.write(header)
    return

def write_payload(file, payload, content, header):
    with open(file, "a") as _file:
        for a in payload:
            content = content.format(*tuple(a))
            _file.write(content + "\n")
            if file == "announcements.csv":
                a = tuple([str(_a)[:29]+"..." if len(str(_a)) > 30 else str(_a) for _a in tuple(a)])
                print(announcement_print.format(*a))
    return
    

def write_to_disk(payload, file):
    header, content = get_csv_head_and_content(file)
    create_file_if_not_existing(file, header)
    write_payload(file, payload, content, header)

    
def handle_announcements(payload):
    announcements=[]
    for announcement in payload:
        a = announcement["args"]
        txhash = announcement["transactionHash"].hex()
        blocknr = int(announcement["blockNumber"])
        tx = w3.eth.get_transaction(txhash)
        sender = tx["from"]
        announcements.append(
            (
                a["schemeId"], 
                a["stealthAddress"].lower(), 
                a["ephemeralPubKey"].hex(), 
                a["metadata"].hex(),
                blocknr,
                sender.lower()
            )
        )
    return announcements

def handle_stakers(payload):
    stakers=[]
    for stakingTx in payload:
        st = stakingTx["args"]
        blocknr = int(stakingTx["blockNumber"])
        stakers.append(
            (
                st["account"].lower(),
                st["totalStaked"],
                blocknr
            )
        )
    return stakers

def handle_withdrawers(payload):
    withdrawers=[]
    for withdrawalTx in payload:
        wd = withdrawalTx["args"]
        blocknr = int(withdrawalTx["blockNumber"])
        withdrawers.append(
            (
                wd["account"].lower(),
                wd["withdrawAddress"].lower(),
                wd["amount"],
                blocknr
            )
        )
    return withdrawers

def start_optimizer(found):
    if found > 0:
        threading.Thread(target=optimize_and_upload).start()
    return

def wait(found):
    if found > 0:
        time.sleep(1)
    else:
        #print(now(), "nothing found", end="\r")
        time.sleep(5)
        #print(" ".join([" "]*50), end="\r")
        
def run():
    global w3, client
    w3, client = init()
    abi = load_contract_abi()
    bytecode = load_contract_bytecode()
    messenger = w3.eth.contract(abi=abi, bytecode=bytecode)
    filters = create_filters(messenger)
    print(announcement_print.format(
            "id", 
            "stealth address",
            "ephemeral pubkey",
            "metadata",
            "blocknr",
            "sender"
        ))
    print("".join(["-"]*155))
    while True:
        found = 0
        newAnnouncements = list(set(filters[0].get_new_entries()))
        if len(newAnnouncements) > 0:
            #print(f"{len(newAnnouncements)} announcements found:")
            newAnnouncements = handle_announcements(newAnnouncements)
            write_to_disk(newAnnouncements, "announcements.csv")
            found = 1
            time.sleep(0.5)

        newStakers = filters[1].get_new_entries()
        if len(newStakers) > 0: 
            newStakers = handle_stakers(newStakers)
            write_to_disk(newStakers, "stakers.csv")
            found = 1
            time.sleep(0.5)

        newWithdrawers = filters[2].get_new_entries()
        if len(newWithdrawers) > 0: 
            newWithdrawers = handle_withdrawers(newWithdrawers)
            write_to_disk(newWithdrawers, "withdrawers.csv")
            found = 1

        start_optimizer(found)
        wait(found)


if __name__ == "__main__":
    try:
        run()
    except KeyboardInterrupt:
        pass